## Import libraries

In [1]:
# data manipulation
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# linear algebra
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
plt.rcParams['figure.figsize'] = 12,6
sns.set_style('darkgrid')
%matplotlib inline

# machine leanring
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, recall_score, precision_score, make_scorer

import math

## Import dataset

In [2]:
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
data_duration = df['Time'].max() / (60*60)
fraud_cost = round(df[df['Class']==1]['Amount'].mean(),2)
number_of_transactions_per_hour = 12 # one transaction per 5 minutes
monitoring_cost = 6780 / (4*40) / number_of_transactions_per_hour # (4*40) means to 40 working hours per week for 4 weeks

print(f'This data is worth {round(data_duration)} hours of transactions.')
print(f'The average fraud cost is RM {fraud_cost}')
print('I am assuming the salary of financial staff in Malaysia is RM 6780.')
print('I am also assuming that one transaction takes 5 minutes to be checked.')
print(f'So, the monitoring cost for one transaction is RM {round(monitoring_cost,2)}')

This data is worth 48 hours of transactions.
The average fraud cost is RM 122.21
I am assuming the salary of financial staff in Malaysia is RM 6780.
I am also assuming that one transaction takes 5 minutes to be checked.
So, the monitoring cost for one transaction is RM 3.53


In [4]:
print(f'For every fraud transaction predicted by our model, monitoring will be done.')
print(f'But if our model predicted a fraud transaction as normal, then we will loss RM {fraud_cost}.')

For every fraud transaction predicted by our model, monitoring will be done.
But if our model predicted a fraud transaction as normal, then we will loss RM 122.21.


## Model building

In [5]:
def create_models():
    models = {}
    models['Logistic Regression'] = LogisticRegression(max_iter=5000)
    models['K-Nearest Neighbors'] = KNeighborsClassifier()
    models['Support Vector Machines'] = SVC()
    models['Decision Tree'] = DecisionTreeClassifier()
    models['Naive Bayes'] = GaussianNB()
    models['Random Forest'] = RandomForestClassifier()
    return models

In [6]:
def evaluate_model(model, X, y, metrics='accuracy'):
    scaler = StandardScaler()
    pipeline = Pipeline(steps=[('scaler', scaler), ('model', model)])
    cv = StratifiedKFold(5, shuffle=True, random_state=123)
    scores = cross_validate(pipeline, X, y, cv=cv, scoring=metrics)
    return scores

In [7]:
# metrics
def false_positive(estimator, X, y):
    predictions = estimator.predict(X)
    cm = confusion_matrix(y,predictions)
    return cm[0][1]

def false_negative(estimator, X, y):
    predictions = estimator.predict(X)
    cm = confusion_matrix(y,predictions)
    return cm[1][0]

def true_positive(estimator, X, y):
    predictions = estimator.predict(X)
    cm = confusion_matrix(y,predictions)
    return cm[1][1]

In [8]:
df = df.iloc[:,1:]
X = df.drop('Class', axis=1)
y = df['Class']

names = []
results = []

scoring = {'fp':false_positive,
           'fn':false_negative,
           'tp':true_positive,
           'recall': 'recall',
           'precision': 'precision'}

for name, model in create_models().items():
    print(f'--- Initializing {name} ---')
    scores = evaluate_model(model, X, y, scoring)
    names.append(name)
    results.append(scores)
    print(f'--- {name} done! ---')

--- Initializing Logistic Regression ---
--- Logistic Regression done! ---
--- Initializing K-Nearest Neighbors ---
--- K-Nearest Neighbors done! ---
--- Initializing Support Vector Machines ---
--- Support Vector Machines done! ---
--- Initializing Decision Tree ---
--- Decision Tree done! ---
--- Initializing Naive Bayes ---
--- Naive Bayes done! ---
--- Initializing Random Forest ---
--- Random Forest done! ---


In [9]:
test_fp = []
test_fn = []
test_tp = []
test_recall = []
test_precision = []
time_taken = []

for i in range(len(results)):
    test_fp.append(results[i]['test_fp'].mean())

for i in range(len(results)):
    test_fn.append(results[i]['test_fn'].mean())
    
for i in range(len(results)):
    test_tp.append(results[i]['test_tp'].mean())
    
for i in range(len(results)):
    test_recall.append(results[i]['test_recall'].mean())
    
for i in range(len(results)):
    test_precision.append(results[i]['test_precision'].mean())
    
for i in range(len(results)):
    time_taken.append(round(results[i]['score_time'].sum(),1) + round(results[i]['fit_time'].sum(),1))

test_fn = pd.Series(test_fn,name='False negative')
test_fp = pd.Series(test_fp,name='False positive')
test_tp = pd.Series(test_tp,name='True positive')
test_recall = pd.Series(test_recall,name='Recall')
test_precision = pd.Series(test_precision,name='Precision')
time_taken = pd.Series(time_taken,name='Time to compute (seconds)')

df_result = pd.concat([test_fp,test_fn,test_tp,test_recall,test_precision,time_taken], axis=1)
df_result.index = names
df_result['F1-score'] = (2 * df_result['Precision'] * df_result['Recall']) / (df_result['Precision'] + df_result['Recall'])
df_result['Cost function (RM)'] = round(df_result['False negative'] * fraud_cost + ((df_result['True positive'] + df_result['False positive']) * monitoring_cost),2)
df_result = df_result.iloc[:,[0,1,2,3,4,6,7,5]]
df_result.sort_values(['Cost function (RM)','F1-score'])

,False positive,False negative,True positive,Recall,Precision,F1-score,Cost function (RM),Time to compute (seconds)
Random Forest,3.8,21.8,76.6,0.778479,0.952508,0.856745,2948.09,1088.4
K-Nearest Neighbors,6.0,22.4,76.0,0.772356,0.927506,0.842851,3027.07,6700.3
Decision Tree,21.2,23.8,74.6,0.758009,0.780282,0.768984,3246.89,86.7
Support Vector Machines,4.0,33.4,65.0,0.660482,0.941996,0.776512,4325.47,1847.4
Logistic Regression,9.2,37.4,61.0,0.619955,0.867879,0.723260,4818.55,14.6
Naive Bayes,1244.0,16.8,81.6,0.829293,0.061737,0.114919,6734.15,3.8


## Borderline-SMOTE SVM

In [10]:
from imblearn.over_sampling import SVMSMOTE

borderline_svm = SVMSMOTE()
X_svm,y_svm = borderline_svm.fit_resample(X,y)

In [11]:
tree_scoring = {'recall':'recall',
                'precision':'precision',
                'f1-score':'f1'}

tree_scores = evaluate_model(DecisionTreeClassifier(), X_svm, y_svm, tree_scoring)

In [12]:
tree_recall = math.floor(tree_scores['test_recall'].mean() * 10 ** 5) / 10 ** 5
tree_precision = math.floor(tree_scores['test_precision'].mean() * 10 ** 5) / 10 ** 5
tree_f1 = math.floor(tree_scores['test_f1-score'].mean() * 10 ** 5) / 10 ** 5

In [14]:
print(f'Recall: {tree_recall * 100:9} %')
print(f'Precision: {tree_precision * 100:.3f} %')
print(f'F1-score: {tree_f1 * 100:7} %')

Recall:    99.958 %
Precision: 99.915 %
F1-score:  99.937 %
